# Проект по SQL сервиса для чтения книг по подписке

**Цель -  проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта**

# План исследования:

**1.  Первичное знакомство;**


  
**Таблица `books`**

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**2. Преобработка данных;**

**3. Посчитать, сколько книг вышло после 1 января 2000 года;**

**4. Для каждой книги посчитать количество обзоров и среднюю оценку;**

**5. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;**

**6. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**

**7. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#сформируем sql запрос
books = pd.io.sql.read_sql(''' SELECT * FROM books''', con = engine)

In [3]:
authors = pd.io.sql.read_sql(''' SELECT * FROM authors''', con = engine)

In [4]:
publishers = pd.io.sql.read_sql(''' SELECT * FROM publishers''', con = engine)

In [5]:
ratings = pd.io.sql.read_sql(''' SELECT * FROM ratings''', con = engine)

In [6]:
reviews = pd.io.sql.read_sql(''' SELECT * FROM reviews''', con = engine)

In [7]:
#посмотрим таблицы
table = books, authors, publishers, ratings, reviews

for row in table:
    display(row.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Предобработка данных

In [8]:
#посмотрим информацию по колонкам
table = books, authors, publishers, ratings, reviews

for row in table:
    row.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        

In [9]:
#преобразуем дату в date
books['publication_date'] = pd.to_datetime(books['publication_date'])

In [10]:
#проверим на пропуски
table = books, authors, publishers, ratings, reviews

for row in table:
    print(row.isna().sum())

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64
author_id    0
author       0
dtype: int64
publisher_id    0
publisher       0
dtype: int64
rating_id    0
book_id      0
username     0
rating       0
dtype: int64
review_id    0
book_id      0
username     0
text         0
dtype: int64


пропусков нет

In [11]:
#проверим на дубликаты
table = books, authors, publishers, ratings, reviews

for row in table:
    print('Кол-во дубликатов -', row.duplicated().sum())

Кол-во дубликатов - 0
Кол-во дубликатов - 0
Кол-во дубликатов - 0
Кол-во дубликатов - 0
Кол-во дубликатов - 0


Дубликатов нет

In [12]:
#проверим на аномальные значения
table = books, authors, publishers, ratings, reviews

for row in table:
    print(row.describe())

           book_id    author_id   num_pages  publisher_id
count  1000.000000  1000.000000  1000.00000   1000.000000
mean    500.500000   320.417000   389.11100    171.270000
std     288.819436   181.620172   229.39014     99.082685
min       1.000000     1.000000    14.00000      1.000000
25%     250.750000   162.750000   249.00000     83.000000
50%     500.500000   316.500000   352.00000    177.500000
75%     750.250000   481.000000   453.00000    258.000000
max    1000.000000   636.000000  2690.00000    340.000000
        author_id
count  636.000000
mean   318.500000
std    183.741666
min      1.000000
25%    159.750000
50%    318.500000
75%    477.250000
max    636.000000
       publisher_id
count    340.000000
mean     170.500000
std       98.293777
min        1.000000
25%       85.750000
50%      170.500000
75%      255.250000
max      340.000000
         rating_id      book_id       rating
count  6456.000000  6456.000000  6456.000000
mean   3228.500000   510.574195     3.928284
s

**Аномалий не замечено, рейтиниги разумные, ревью тоже. Данные в хорошем состоянии достались нам**

# Cколько книг вышло после 1 января 2000 года

In [13]:
#посчитаем, сколько книг вышло после 1 января 2000 года
task_one = ''' 
SELECT COUNT(book_id) 
FROM books
WHERE publication_date > '2000-01-01'
'''
task_one = pd.io.sql.read_sql(task_one, con = engine)

task_one.iloc[0,0]

819

**После 1 января 2000 года вышло 819 книг**

# Посчитаем количество обзоров и среднюю оценку для каждой из книг

In [14]:
#для каждой книги посчитаем количество обзоров и среднюю оценку

task_two = '''
WITH rating AS (
        SELECT book_id,
        ROUND(AVG(rating), 2) AS avg_rating
        FROM ratings 
        GROUP BY book_id),

review AS (
        SELECT book_id,
        COUNT(review_id) AS cnt_reviews
        FROM reviews 
        GROUP BY book_id)

SELECT b.title,
       b.book_id,
       r.avg_rating,
       re.cnt_reviews
FROM books AS b 
LEFT JOIN rating AS r ON b.book_id=r.book_id
LEFT JOIN review AS re ON b.book_id=re.book_id
'''

task_two = pd.io.sql.read_sql(task_two, con = engine)

task_two.sort_values(by=['avg_rating', 'cnt_reviews'], ascending=False)

,title,book_id,avg_rating,cnt_reviews
911,A Dirty Job (Grim Reaper #1),17,5.00,4.0
320,Moneyball: The Art of Winning an Unfair Game,444,5.00,3.0
648,School's Out—Forever (Maximum Ride #2),553,5.00,3.0
28,Light in August,390,5.00,2.0
44,Act of Treason (Mitch Rapp #9),57,5.00,2.0
...,...,...,...,...
446,The World Is Flat: A Brief History of the Twen...,915,2.25,3.0
900,Drowning Ruth,202,2.00,3.0
711,His Excellency: George Washington,316,2.00,2.0
960,Junky,371,2.00,2.0


**Вывели для каждой книги количество обзоров и среднюю оценку**

- **Топ-5** по рейтингу:

  - A Dirty Job (Grim Reaper #1) 5.00
  - Moneyball: The Art of Winning an Unfair Game 5.00
  - School's Out—Forever (Maximum Ride #2) 5.00
  - Light in August 5.00
  - Act of Treason (Mitch Rapp #9) 5.00

# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

<b>С помощью данного кода мы сможем исключить из анализа брошюры</b>

In [15]:
#найдем издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

task_free = '''
WITH book AS (
            SELECT publisher_id,
            COUNT(book_id) AS cnt_book
            FROM books
            WHERE num_pages > 50
            GROUP BY publisher_id)

SELECT publisher
FROM book AS b
JOIN publishers AS pub ON b.publisher_id=pub.publisher_id
ORDER BY cnt_book DESC
LIMIT 1
'''
task_free = pd.io.sql.read_sql(task_free, con = engine)

task_free.iloc[0,0]

'Penguin Books'

**Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books**

# Определим автора с самой высокой средней оценкой книг

In [16]:
#определим автора с самой высокой средней оценкой книг, учтем только книги с 50 и более оценкам
task_four = '''
WITH rating AS (
                SELECT book_id,
                COUNT(rating_id) AS cnt_rating_id,
                ROUND(AVG(rating), 2) AS avg_rating
                FROM ratings
                GROUP BY book_id
                HAVING COUNT(rating_id) >= 50
                ORDER BY avg_rating DESC)
       
SELECT aut.author,
       ROUNd(AVG(aut.avg_rating), 2) as avg_rating_final
FROM  (SELECT author, 
              avg_rating
       FROM books AS b
       JOIN rating AS rat ON b.book_id=rat.book_id
       JOIN authors AS a ON b.author_id=a.author_id) AS aut
GROUP BY aut.author
LIMIT 1
'''
task_four = pd.io.sql.read_sql(task_four, con = engine)

task_four.iloc[0,0]

'J.K. Rowling/Mary GrandPré'

**Автор с самой высокой средней оценкой книг - JK Rowling/Mary GrandPré**

# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [17]:
#посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

task_five = '''
WITH rating AS(
            SELECT username,
            COUNT(rating_id) AS cnt_rating_id
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50)

SELECT ROUND(AVG(us.cnt_review_id),2)
FROM  (SELECT rev.username,
              COUNT(review_id) AS cnt_review_id
       FROM rating AS rat
       JOIN reviews AS rev ON rat.username = rev.username
       GROUP BY rev.username) AS us
'''

task_five = pd.io.sql.read_sql(task_five, con = engine)

task_five.iloc[0,0]

24.33

**Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,33**

**Итоговые выводы:**

 - После 1 января 2000 года было выпущено 819 книг.

 - **Топ-5** по рейтингу:

  - A Dirty Job (Grim Reaper #1) 5.00
  - Moneyball: The Art of Winning an Unfair Game 5.00
  - School's Out—Forever (Maximum Ride #2) 5.00
  - Light in August 5.00
  - Act of Treason (Mitch Rapp #9) 5.00
  
 - Издательство Penguin Books - выпустило наибольше число книг толще 50 страниц.

 - Автор с самой высокой средней оценкой книг - JK Rowling/Mary GrandPré.

 - Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,33.